# Keras MobileNet Benchmark
(Note: Forked from Beluga : [https://www.kaggle.com/gaborfodor/greyscale-mobilenet-lb-0-892])

In a previous benchmark we used a simple three layer ConvNet. This time we use a deeper MobileNet architecture on greyscale strokes. 

This kernel has three main components:

* MobileNet
* Fast and memory efficient Image Generator with temporal 
* Full training & submission with Kaggle Kernel

I did some paramer search but it should not be hard to improve the current score.

## Setup
Import the necessary libraries and a few helper functions.

In [2]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import pickle as pkl
start = dt.datetime.now()

In [3]:
sub_bear = pd.read_csv("bear_mn_submission_4857.csv")
sub_best = pd.read_csv("sub_bus_replacement.csv")


In [4]:
new_sub = pd.merge(sub_bear, sub_best, on='key_id')

In [9]:
new_sub

,key_id,word_x,word_y,0,1
0,9000003627287624,panda teddy-bearbear,radio stereo power_outlet,radio,stereo power_outlet
1,9000010688666847,panda teddy-bearbear,hockey_puck bottlecap pool,hockey_puck,bottlecap pool
2,9000023642890129,panda teddy-bearbear,The_Great_Wall_of_China castle camel,The_Great_Wall_of_China,castle camel
3,9000038588854897,panda teddy-bearbear,mountain tent finger,mountain,tent finger
4,9000052667981386,panda teddy-bearbear,fireplace campfire fire_hydrant,fireplace,campfire fire_hydrant
5,9000057427034623,panda teddy-bearbear,fence jail spreadsheet,fence,jail spreadsheet
6,9000065506980882,panda teddy-bearbear,wine_glass hourglass wine_bottle,wine_glass,hourglass wine_bottle
7,9000087586309806,panda teddy-bearbear,submarine swan dolphin,submarine,swan dolphin
8,9000092580281382,panda teddy-bearbear,bracelet hand wristwatch,bracelet,hand wristwatch
9,9000096661653918,panda teddy-bearbear,hourglass vase peanut,hourglass,vase peanut


In [7]:
new_sub = pd.concat([new_sub,new_sub['word_y'].str.split(' ', n = 1, expand = True) ], axis=1)

In [ ]:
k1 = new_sub[(new_sub['word11'] == 'bear')&((new_sub['word21'] == 'teddy-bear')||(new_sub['word21'] == 'panda'))]['key_id']
k2 = new_sub[(new_sub['word11'] == 'teddy-bear')&((new_sub['word21'] == 'bear')||(new_sub['word21'] == 'panda'))]['key_id']
k3 = new_sub[(new_sub['word11'] == 'panda')&((new_sub['word21'] == 'bear')||(new_sub['word21'] == 'teddy-bear'))]['key_id']

In [ ]:
sub.loc[sub['key_id'].isin(k1),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('bear teddy-bear', 'teddy-bear bear')
sub.loc[sub['key_id'].isin(k1),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('bear panda', 'panda bear')

In [ ]:
sub.loc[sub['key_id'].isin(k2),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('teddy-bear bear', 'bear teddy-bear')
sub.loc[sub['key_id'].isin(k2),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('teddy-bear panda', 'panda  teddy-bear')

In [ ]:
sub.loc[sub['key_id'].isin(k3),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('panda bear', 'bear panda')
sub.loc[sub['key_id'].isin(k3),'word'] = sub[sub['key_id'].isin(k)]['word'].str.replace('panda teddy-bear', 'teddy-bear panda')